In [3]:
import json
from typing import TypedDict


class LLMAnswer(TypedDict):
    answer: str
    is_correct: bool

class Question(TypedDict):
    id: str
    question: str
    expected_answer: str
    category: str
    difficulty: str
    llm_answers: dict[str, LLMAnswer]


questions:dict[str, list[Question]] = json.load(open("../frontend/data/sample_trivia.json"))

### Test Xenova/distilbert-base-uncased-finetuned-sst2

In [7]:
# Import required libraries
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification


classifier = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Function to format input and classify
def check_answer(question:str, expected_answer:str, user_answer:str):
    input_text = f"Question: {question} Expected answer: {expected_answer}. User answer: {user_answer}"
    result = classifier(input_text)
    return result[0]

# Test with your example
test_case = {
    "question": "Who has built the Chateau de Versailles?",
    "expected_answer": "Louis XIV",
    "user_answer": "Louis XVI"
}

result = check_answer(
    test_case["question"],
    test_case["expected_answer"],
    test_case["user_answer"]
)
print(result)

Device set to use cpu


{'label': 'NEGATIVE', 'score': 0.9918941855430603}


### Finetune model